In [1]:
import pandas as pd
import numpy as np
import re, json

In [2]:
df = pd.read_json("business.json", lines = True)

In [3]:
### Select top10 Chain Pizza Restaurants
shop_list=["Domino","Pizza hut","Little caesars","Papa John","California pizza kitchen",
           "Papa Murphy","Sbarro","Marco's pizza","Chuck E. cheese","Cici's pizza"]
tem1 = pd.DataFrame(columns=['business_id','name','city','state','stars_x','review_count',
                             'attributes','user_id','stars_y','text','date'])

for a in shop_list:
    tem = df[df['name'].str.contains(a, na=False, case=False)==True]
    tem['name']=a
    tem1 = pd.concat([tem1,tem],axis = 0)
   
PizzaChain = tem1.copy()
PizzaChain.shape[0]

# select Pizza Hut
df1 = PizzaChain[PizzaChain['name'] == "Pizza hut"]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [4]:
df2 = pd.read_csv('sentiment_result.csv')

In [5]:
business_list = pd.merge(left = df2, right = df1, how = 'left', on=['business_id'] )
business_list = business_list[['business_id','attributes']]
business_list = business_list.reset_index().drop("index", axis = 1)
business_list.head()
# business_list.shape[0]

,business_id,attributes
0,kqhHw_9KA30_RYk66mMbJA,"{'Ambience': '{'romantic': False, 'intimate': ..."
1,hWRZaOk4JWUPFhcomdOLwA,"{'NoiseLevel': ''quiet'', 'RestaurantsGoodForG..."
2,6EwA03Jcw5VaPc8BNiwrpw,"{'WiFi': 'u'no'', 'GoodForKids': 'True', 'Busi..."
3,HUEZLPIM64cDZnfHE1H3AA,"{'RestaurantsReservations': 'False', 'HasTV': ..."
4,_zy93iWV8Z-YmfvjWPcfAQ,"{'OutdoorSeating': 'False', 'RestaurantsReserv..."


In [ ]:
# attributes = []
# attr = business_list.loc[0,'attributes'].keys()
# attributes.extend(list(attr))
# attributes

In [6]:
# extract attributes
attributes = []
for ind in business_list.index: 
    tem = business_list.loc[ind, "attributes"].keys()
    attributes.extend(list(tem))

attributes = sorted(list(set(attributes)))


In [7]:
attr_dict = {}
for attr in attributes: 
    attr_dict[attr] = []
for ind in business_list.index:
    attribute = business_list.loc[ind, "attributes"]
    for attr in attributes: 
        if attr in attribute.keys(): 
            attr_dict[attr].append(attribute[attr])
        else: 
            attr_dict[attr].append(np.nan)

In [30]:
attr_df = pd.DataFrame(attr_dict)
pizza_attributes = pd.concat([business_list, attr_df], axis = 1)

In [36]:
#pizza_attributes.head()

In [38]:
na_num = {}
for col in attr_df.columns: 
    na_num[col] = attr_df[col].isnull().sum()
extract_attr = []
for attr in list(na_num.keys()): 
    if na_num[attr] < 70:    #145*0.25 = 29
        extract_attr.append(attr)
extract_attr

['Ambience',
 'BusinessAcceptsCreditCards',
 'BusinessParking',
 'Caters',
 'GoodForKids',
 'HasTV',
 'OutdoorSeating',
 'RestaurantsAttire',
 'RestaurantsDelivery',
 'RestaurantsGoodForGroups',
 'RestaurantsPriceRange2',
 'RestaurantsReservations',
 'RestaurantsTakeOut']

In [39]:
pizza_attributes = pizza_attributes[["business_id",
 'BusinessAcceptsCreditCards',
 'Caters',
 'GoodForKids',
 'HasTV',
 'OutdoorSeating',
 'RestaurantsAttire',
 'RestaurantsDelivery',
 'RestaurantsGoodForGroups',
 'RestaurantsPriceRange2',
 'RestaurantsReservations',
 'RestaurantsTakeOut']]

In [40]:
pizza_attributes

,business_id,BusinessAcceptsCreditCards,Caters,GoodForKids,HasTV,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTakeOut
0,kqhHw_9KA30_RYk66mMbJA,NaN,True,False,False,False,NaN,True,True,2,False,True
1,hWRZaOk4JWUPFhcomdOLwA,True,False,True,True,False,u'casual',True,True,1,True,True
2,6EwA03Jcw5VaPc8BNiwrpw,True,False,True,True,False,u'casual',True,False,1,False,True
3,HUEZLPIM64cDZnfHE1H3AA,True,False,True,True,True,u'casual',True,True,1,False,True
4,_zy93iWV8Z-YmfvjWPcfAQ,NaN,NaN,NaN,NaN,False,NaN,True,NaN,1,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...
140,koPEEsp5fVr5_l_vYYlNeA,True,True,False,True,False,u'casual',True,True,1,False,True
141,kyKSPGalAGSUdEv38Cx9Uw,NaN,NaN,NaN,True,None,u'casual',True,True,2,False,True
142,Sb9FAzrW_NGA3BiViZDLIw,True,False,True,True,False,'casual',True,True,1,False,True
143,Gjhp2wjWeMrCbRsirVIYJw,True,False,True,True,False,'casual',True,False,1,False,True


In [29]:
pizza_attributes

,business_id,Ambience,BusinessAcceptsCreditCards,BusinessParking,Caters,GoodForKids,HasTV,OutdoorSeating,RestaurantsAttire,RestaurantsDelivery,RestaurantsGoodForGroups,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsTakeOut
0,kqhHw_9KA30_RYk66mMbJA,"{'romantic': False, 'intimate': False, 'classy...",NaN,"{'garage': False, 'street': False, 'validated'...",True,False,False,False,NaN,True,True,2,False,True
1,hWRZaOk4JWUPFhcomdOLwA,"{'romantic': False, 'intimate': False, 'classy...",True,"{'garage': False, 'street': False, 'validated'...",False,True,True,False,u'casual',True,True,1,True,True
2,6EwA03Jcw5VaPc8BNiwrpw,"{'romantic': False, 'intimate': False, 'classy...",True,"{'garage': False, 'street': False, 'validated'...",False,True,True,False,u'casual',True,False,1,False,True
3,HUEZLPIM64cDZnfHE1H3AA,"{'romantic': False, 'intimate': False, 'touris...",True,"{'garage': False, 'street': False, 'validated'...",False,True,True,True,u'casual',True,True,1,False,True
4,_zy93iWV8Z-YmfvjWPcfAQ,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,True,NaN,1,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,koPEEsp5fVr5_l_vYYlNeA,NaN,True,"{'garage': False, 'street': False, 'validated'...",True,False,True,False,u'casual',True,True,1,False,True
141,kyKSPGalAGSUdEv38Cx9Uw,"{'touristy': False, 'hipster': False, 'romanti...",NaN,None,NaN,NaN,True,None,u'casual',True,True,2,False,True
142,Sb9FAzrW_NGA3BiViZDLIw,"{'touristy': False, 'hipster': False, 'romanti...",True,"{'garage': False, 'street': False, 'validated'...",False,True,True,False,'casual',True,True,1,False,True
143,Gjhp2wjWeMrCbRsirVIYJw,None,True,NaN,False,True,True,False,'casual',True,False,1,False,True


In [41]:
pizza_hut = pd.merge(left = df2, right = pizza_attributes, how = 'left', on=['business_id'] )

In [ ]:
pizza_hut

In [42]:
pizza_hut.to_csv('pizza_hut.csv')